Import libraries

In [1]:
from copy import copy
import pandas as pd
from ib_insync import *
util.startLoop()


Connect to TWS (paper account: 7497, live account 7496)

In [ ]:
#ib.isConnected()
#ib.disconnect()

In [3]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=13)
# util.logToConsole()
ib.isConnected()

True

Read isins to trade

In [4]:
#isin_init_dt = pd.read_csv('ISIN_INDU_IB.csv')
isin_init_dt = pd.read_csv('ISIN_CAC_IB.csv')
#print( isin_init_dt.head() )
isin_init_dt.tail(3)

,Ticker,ISIN,LONG_COMP_NAME,CRNCY,PX_LAST,PRIMARY_EXCHANGE_NAME,EQY_PRIM_EXCH_SHRT
37,RI FP Equity,FR0000120693,Pernod Ricard SA,EUR,193.400,EN Paris,FP
38,STM FP Equity,NL0000226223,STMicroelectronics NV,EUR,40.515,EN Paris,FP
39,WLN FP Equity,FR0011981968,Worldline SA/France,EUR,44.320,EN Paris,FP


Choose only isins with data permission (traded in Paris)

In [5]:
isin_init_dt = isin_init_dt.loc[ isin_init_dt["EQY_PRIM_EXCH_SHRT"] == "FP" ]
isin_init_dt.reset_index(drop=True, inplace=True)
isin_init_dt.head()

,Ticker,ISIN,LONG_COMP_NAME,CRNCY,PX_LAST,PRIMARY_EXCHANGE_NAME,EQY_PRIM_EXCH_SHRT
0,OR FP Equity,FR0000120321,L'Oreal SA,EUR,376.600,EN Paris,FP
1,DG FP Equity,FR0000125486,Vinci SA,EUR,102.880,EN Paris,FP
2,STLA FP Equity,NL00150001Q9,Stellantis NV,EUR,16.982,EN Paris,FP
3,TTE FP Equity,FR0000120271,TotalEnergies SE,EUR,51.890,EN Paris,FP
4,AI FP Equity,FR0000120073,Air Liquide SA,EUR,147.760,EN Paris,FP


In [6]:
#contract = Contract(secIdType='ISIN', secId='US0378331005', exchange='SMART', currency='USD')
#ib.qualifyContracts(contract)

In [7]:
isin_init_dt["ISIN"].tail()

33    FR0010307819
34    FR0000045072
35    FR0000120693
36    NL0000226223
37    FR0011981968
Name: ISIN, dtype: object

Create and qualify contracts for each isin

In [8]:
isin_init_dt["contracts"] = [ Contract(secIdType='ISIN', secId=isin_input, exchange='SBF', currency='EUR') for isin_input in isin_init_dt["ISIN"] ]
ib.qualifyContracts(*isin_init_dt["contracts"])

isin_init_dt.head()

,Ticker,ISIN,LONG_COMP_NAME,CRNCY,PX_LAST,PRIMARY_EXCHANGE_NAME,EQY_PRIM_EXCH_SHRT,contracts
0,OR FP Equity,FR0000120321,L'Oreal SA,EUR,376.600,EN Paris,FP,"Contract(secType='STK', conId=29612116, symbol..."
1,DG FP Equity,FR0000125486,Vinci SA,EUR,102.880,EN Paris,FP,"Contract(secType='STK', conId=29612226, symbol..."
2,STLA FP Equity,NL00150001Q9,Stellantis NV,EUR,16.982,EN Paris,FP,"Contract(secType='STK', conId=463647471, symbo..."
3,TTE FP Equity,FR0000120271,TotalEnergies SE,EUR,51.890,EN Paris,FP,"Contract(secType='STK', conId=29612193, symbol..."
4,AI FP Equity,FR0000120073,Air Liquide SA,EUR,147.760,EN Paris,FP,"Contract(secType='STK', conId=29612135, symbol..."


Extract contract id from isin and add to data table

In [9]:
isin_init_dt['conId'] = 0

for i in range( isin_init_dt.shape[0] ):
    isin_init_dt['conId'][i] = isin_init_dt['contracts'][i].conId

<ipython-input-9-fa670773c92e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isin_init_dt['conId'][i] = isin_init_dt['contracts'][i].conId


Copy table to new table: Trade execution can be run from here forward multiple times

In [10]:
isin_dt = copy( isin_init_dt )

Get market data

In [11]:
#contract_input = isin_dt["contracts"][0]
market_data_dt = util.df( ib.reqHistoricalData(
                contract = isin_dt["contracts"][0],
                endDateTime='', #'20220207 00:00:00',
                durationStr= '2 D', #600 S',
                barSizeSetting='1 day', #10 mins',# '1 day',
                whatToShow='MIDPOINT',
                useRTH=False,
                formatDate=1,
                keepUpToDate=False) )

In [12]:
market_data_dt

,date,open,high,low,close,volume,average,barCount
0,2022-02-21,360.75,361.950,345.175,346.575,-1.0,-1.0,-1
1,2022-02-22,346.45,357.775,340.100,348.725,-1.0,-1.0,-1


Choose return: 
1) Close to close
2) Open to close

In [18]:
# 1) Close to close
#market_data_dt['close'].pct_change(1).tail()[1]
market_data_dt['close'].pct_change(1).tail(2)[1]

# 2) Open to close
#( (market_data_dt['close'] / market_data_dt['open']) -1 ).tail(1)[1]

0.006203563442256366

In [19]:
isin_dt['Return'] = 0.0
isin_dt['Return'][0]


0.0

In [20]:
isin_dt['PX_LAST'] = 0.0
isin_dt['PX_LAST'][0]


0.0

In [21]:
for i in range( isin_dt.shape[0]):
    market_data_tmp_dt = util.df( ib.reqHistoricalData(
                contract = isin_dt["contracts"][i],
                endDateTime='', #'20220207 00:00:00',
                durationStr= '2 D', #600 S',
                barSizeSetting='1 day', #10 mins',# '1 day',
                whatToShow='MIDPOINT',
                useRTH=False,
                formatDate=1,
                keepUpToDate=False)) 
    isin_dt['PX_LAST'][i] = market_data_tmp_dt['close'].tail(1)[1]            
    isin_dt['Return'][i] = market_data_tmp_dt['close'].pct_change().tail(1)[1]
    #isin_dt['Return'][i] = market_data_tmp_dt['close'].pct_change(1).tail(1)[1]
    #isin_dt['Return'][i] = ( ( market_data_tmp_dt['close'] / market_data_tmp_dt['open'] ) -1 ).tail(1)[1]
    print( isin_dt['ISIN'][i], isin_dt['PX_LAST'][i] , isin_dt['Return'][i] )
                

<ipython-input-21-e884176b57cc>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isin_dt['PX_LAST'][i] = market_data_tmp_dt['close'].tail(1)[1]
<ipython-input-21-e884176b57cc>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isin_dt['Return'][i] = market_data_tmp_dt['close'].pct_change().tail(1)[1]


FR0000120321 348.725 0.006203563442256366
FR0000125486 97.81 -0.015649373521863796
NL00150001Q9 16.334 0.001717159327854656
FR0000120271 49.8875 -0.001101266456424832
FR0000120073 146.48 -0.0014315904287954817
FR0014003TT8 40.325 0.011412089290193217
FR0000120628 26.2975 -0.010162792886044936
FR0014000MR3 86.205 0.0
FR0000131104 60.665 -0.006387683236426134
FR0000120644 54.575 -0.013912729243834021
FR0000125338 184.925 0.014956092206366733
FR0000120172 17.9575 0.0063042869151022085
FR0000125007 60.235 -0.007660626029654116
FR0000121667 162.28 -0.01078939347759833
FR0000127771 11.33 0.001325673884224443
FR0000121014 669.15 -0.004611379695053985
FR0000121261 130.425 -0.015474617852424877
FR0000121485 643.25 -0.013268906273968373
FR0000130577 61.12 -0.009560849133041671
FR0000131906 33.5725 -0.03769258330347558
FR0000073272 113.17 0.019457706512926665
FR0000121329 83.47 0.0048152160828218715
FR0000051807 324.0 0.03398755385351837
FR0000052292 1207.75 0.0066680558449676575
FR0010208488 14.

In [22]:
isin_dt.head()


,Ticker,ISIN,LONG_COMP_NAME,CRNCY,PX_LAST,PRIMARY_EXCHANGE_NAME,EQY_PRIM_EXCH_SHRT,contracts,conId,Return
0,OR FP Equity,FR0000120321,L'Oreal SA,EUR,348.7250,EN Paris,FP,"Contract(secType='STK', conId=29612116, symbol...",29612116,0.006204
1,DG FP Equity,FR0000125486,Vinci SA,EUR,97.8100,EN Paris,FP,"Contract(secType='STK', conId=29612226, symbol...",29612226,-0.015649
2,STLA FP Equity,NL00150001Q9,Stellantis NV,EUR,16.3340,EN Paris,FP,"Contract(secType='STK', conId=463647471, symbo...",463647471,0.001717
3,TTE FP Equity,FR0000120271,TotalEnergies SE,EUR,49.8875,EN Paris,FP,"Contract(secType='STK', conId=29612193, symbol...",29612193,-0.001101
4,AI FP Equity,FR0000120073,Air Liquide SA,EUR,146.4800,EN Paris,FP,"Contract(secType='STK', conId=29612135, symbol...",29612135,-0.001432


In [23]:
isin_dt['quantile'] = pd.qcut(isin_dt['Return'], q=5, labels=False)
isin_dt.sort_values('quantile', inplace=True)

In [24]:
# Definition for Reversal
#isin_dt['Trade'] = 'NONE'
#isin_dt.loc[ isin_dt['quantile'] == 4 , 'Trade' ] = 'SELL'
#isin_dt.loc[ isin_dt['quantile'] == 0 , 'Trade' ] = 'BUY'

isin_dt['DIRECTION'] = 'NONE'
isin_dt.loc[ isin_dt['quantile'] == 4 , 'DIRECTION' ] = 'SHORT'
isin_dt.loc[ isin_dt['quantile'] == 0 , 'DIRECTION' ] = 'LONG'

In [25]:
isin_dt.head(3)

,Ticker,ISIN,LONG_COMP_NAME,CRNCY,PX_LAST,PRIMARY_EXCHANGE_NAME,EQY_PRIM_EXCH_SHRT,contracts,conId,Return,quantile,DIRECTION
1,DG FP Equity,FR0000125486,Vinci SA,EUR,97.8100,EN Paris,FP,"Contract(secType='STK', conId=29612226, symbol...",29612226,-0.015649,0,LONG
30,GLE FP Equity,FR0000130809,Societe Generale SA,EUR,29.8575,EN Paris,FP,"Contract(secType='STK', conId=29612232, symbol...",29612232,-0.049805,0,LONG
19,RNO FP Equity,FR0000131906,Renault SA,EUR,33.5725,EN Paris,FP,"Contract(secType='STK', conId=29612183, symbol...",29612183,-0.037693,0,LONG


In [26]:
isin_dt['Notional_Total'] = 1000000
# Maybe replace with portfolio value instead of fixed number

In [27]:
#isin_dt.groupby(['Trade']).size()['BUY']

In [28]:
isin_dt['Notional_Target_Pos'] = 0
isin_dt.loc[ isin_dt['DIRECTION'] == 'SHORT' , 'Notional_Target_Pos' ] =  - isin_dt['Notional_Total'] / isin_dt.groupby(['DIRECTION']).size()['SHORT']

In [29]:
isin_dt.loc[ isin_dt['DIRECTION'] == 'LONG' , 'Notional_Target_Pos' ] =  isin_dt['Notional_Total'] / isin_dt.groupby(['DIRECTION']).size()['LONG']

In [30]:
isin_dt["Target_Stk"] = round( isin_dt["Notional_Target_Pos"] /  isin_dt["PX_LAST"] )

In [31]:
positions_initial_dt = util.df( ib.positions() )

positions_initial_dt['conId'] = 0

for i in range( positions_initial_dt.shape[0] ):
    positions_initial_dt['conId'][i] = positions_initial_dt['contract'][i].conId


<ipython-input-31-4b27c720a1d5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positions_initial_dt['conId'][i] = positions_initial_dt['contract'][i].conId


In [32]:
isin_dt.head(3)

,Ticker,ISIN,LONG_COMP_NAME,CRNCY,PX_LAST,PRIMARY_EXCHANGE_NAME,EQY_PRIM_EXCH_SHRT,contracts,conId,Return,quantile,DIRECTION,Notional_Total,Notional_Target_Pos,Target_Stk
1,DG FP Equity,FR0000125486,Vinci SA,EUR,97.8100,EN Paris,FP,"Contract(secType='STK', conId=29612226, symbol...",29612226,-0.015649,0,LONG,1000000,125000.0,1278.0
30,GLE FP Equity,FR0000130809,Societe Generale SA,EUR,29.8575,EN Paris,FP,"Contract(secType='STK', conId=29612232, symbol...",29612232,-0.049805,0,LONG,1000000,125000.0,4187.0
19,RNO FP Equity,FR0000131906,Renault SA,EUR,33.5725,EN Paris,FP,"Contract(secType='STK', conId=29612183, symbol...",29612183,-0.037693,0,LONG,1000000,125000.0,3723.0


In [33]:
isin_exec_dt = pd.merge( isin_dt.loc[:, isin_dt.columns != 'position' ], positions_initial_dt[['conId', 'position']], on='conId', how='left')



In [34]:
#isin_dt.drop(['position_x', 'position_y'], axis = 1, inplace=True)

In [35]:
isin_exec_dt.loc[ isin_exec_dt['position'].isna() , 'position' ] = 0

In [36]:
isin_exec_dt['Trade_Stk_Signed'] = round( isin_exec_dt["Target_Stk"] - isin_exec_dt["position"] )

In [37]:
isin_exec_dt['Trade'] = 'NONE'

isin_exec_dt.loc[ isin_exec_dt['Trade_Stk_Signed'] < 0 , 'Trade' ] = 'SELL'
isin_exec_dt.loc[ isin_exec_dt['Trade_Stk_Signed'] > 0 , 'Trade' ] = 'BUY'

In [38]:
isin_exec_dt['Trade_Stk'] = abs( isin_exec_dt['Trade_Stk_Signed'] )

In [39]:
isin_exec_dt


,Ticker,ISIN,LONG_COMP_NAME,CRNCY,PX_LAST,PRIMARY_EXCHANGE_NAME,EQY_PRIM_EXCH_SHRT,contracts,conId,Return,quantile,DIRECTION,Notional_Total,Notional_Target_Pos,Target_Stk,position,Trade_Stk_Signed,Trade,Trade_Stk
0,DG FP Equity,FR0000125486,Vinci SA,EUR,97.8100,EN Paris,FP,"Contract(secType='STK', conId=29612226, symbol...",29612226,-0.015649,0,LONG,1000000,125000.0,1278.0,0.0,1278.0,BUY,1278.0
1,GLE FP Equity,FR0000130809,Societe Generale SA,EUR,29.8575,EN Paris,FP,"Contract(secType='STK', conId=29612232, symbol...",29612232,-0.049805,0,LONG,1000000,125000.0,4187.0,0.0,4187.0,BUY,4187.0
2,RNO FP Equity,FR0000131906,Renault SA,EUR,33.5725,EN Paris,FP,"Contract(secType='STK', conId=29612183, symbol...",29612183,-0.037693,0,LONG,1000000,125000.0,3723.0,3583.0,140.0,BUY,140.0
3,KER FP Equity,FR0000121485,Kering SA,EUR,643.2500,EN Paris,FP,"Contract(secType='STK', conId=29612178, symbol...",29612178,-0.013269,0,LONG,1000000,125000.0,194.0,0.0,194.0,BUY,194.0
4,CS FP Equity,FR0000120628,AXA SA,EUR,26.2975,EN Paris,FP,"Contract(secType='STK', conId=29612141, symbol...",29612141,-0.010163,0,LONG,1000000,125000.0,4753.0,-4705.0,9458.0,BUY,9458.0
5,ML FP Equity,FR0000121261,Cie Generale des Etablissements Michelin SCA,EUR,130.4250,EN Paris,FP,"Contract(secType='STK', conId=29612156, symbol...",29612156,-0.015475,0,LONG,1000000,125000.0,958.0,0.0,958.0,BUY,958.0
6,BN FP Equity,FR0000120644,Danone SA,EUR,54.5750,EN Paris,FP,"Contract(secType='STK', conId=29612111, symbol...",29612111,-0.013913,0,LONG,1000000,125000.0,2290.0,-2259.0,4549.0,BUY,4549.0
7,EL FP Equity,FR0000121667,EssilorLuxottica SA,EUR,162.2800,EN Paris,FP,"Contract(secType='STK', conId=1321740, symbol=...",1321740,-0.010789,0,LONG,1000000,125000.0,770.0,0.0,770.0,BUY,770.0
8,PUB FP Equity,FR0000130577,Publicis Groupe SA,EUR,61.1200,EN Paris,FP,"Contract(secType='STK', conId=1254581, symbol=...",1254581,-0.009561,1,NONE,1000000,0.0,0.0,0.0,0.0,NONE,0.0
9,SU FP Equity,FR0000121972,Schneider Electric SE,EUR,139.4300,EN Paris,FP,"Contract(secType='STK', conId=29612196, symbol...",29612196,-0.002076,1,NONE,1000000,0.0,0.0,896.0,-896.0,SELL,896.0


In [40]:
isin_orders_dt = isin_exec_dt.loc[ isin_exec_dt['Trade'] != 'NONE'].reset_index(drop=True)
isin_orders_dt.head(3)

,Ticker,ISIN,LONG_COMP_NAME,CRNCY,PX_LAST,PRIMARY_EXCHANGE_NAME,EQY_PRIM_EXCH_SHRT,contracts,conId,Return,quantile,DIRECTION,Notional_Total,Notional_Target_Pos,Target_Stk,position,Trade_Stk_Signed,Trade,Trade_Stk
0,DG FP Equity,FR0000125486,Vinci SA,EUR,97.8100,EN Paris,FP,"Contract(secType='STK', conId=29612226, symbol...",29612226,-0.015649,0,LONG,1000000,125000.0,1278.0,0.0,1278.0,BUY,1278.0
1,GLE FP Equity,FR0000130809,Societe Generale SA,EUR,29.8575,EN Paris,FP,"Contract(secType='STK', conId=29612232, symbol...",29612232,-0.049805,0,LONG,1000000,125000.0,4187.0,0.0,4187.0,BUY,4187.0
2,RNO FP Equity,FR0000131906,Renault SA,EUR,33.5725,EN Paris,FP,"Contract(secType='STK', conId=29612183, symbol...",29612183,-0.037693,0,LONG,1000000,125000.0,3723.0,3583.0,140.0,BUY,140.0


In [41]:
isin_orders_dt['orders'] = ''

for i in range( isin_orders_dt.shape[0]):
    #isin_orders_dt['orders'][i] = Order(orderType='MKT', action= isin_orders_dt['Trade'][i], totalQuantity = isin_orders_dt['Trade_Stk'][i] , transmit=False) 
    #isin_orders_dt['orders'][i] = Order(orderType='MKT', tif='DAY', action= isin_orders_dt['Trade'][i], totalQuantity = isin_orders_dt['Trade_Stk'][i] , transmit=True) 
    isin_orders_dt['orders'][i] = Order(orderType='MKT',  tif='DAY', action= isin_orders_dt['Trade'][i], totalQuantity = isin_orders_dt['Trade_Stk'][i] , transmit=True) 
    


<ipython-input-41-516719babcb8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isin_orders_dt['orders'][i] = Order(orderType='MKT',  tif='DAY', action= isin_orders_dt['Trade'][i], totalQuantity = isin_orders_dt['Trade_Stk'][i] , transmit=True)


See options for oder timing with tif='' under 
https://www.interactivebrokers.com/en/software/tws/usersguidebook/ordertypes/time_in_force_for_orders.htm
https://interactivebrokers.github.io/tws-api/classIBApi_1_1Order.html#a6b82712a718127487631727db08f67d4


In [42]:
for i in range( isin_orders_dt.shape[0]):
    _placeorder =  ib.placeOrder( isin_orders_dt["contracts"][i], isin_orders_dt["orders"][i] )
    

Peer closed connection.


In [41]:
#_placeorder

In [42]:
#for i in range( isin_orders_dt.shape[0]):
#    _cancelorder =  ib.cancelOrder(  isin_orders_dt["orders"][i] )

Positions

In [43]:
positions_dt = util.df( ib.positions() )

positions_dt['conId'] = 0

for i in range( positions_dt.shape[0] ):
    positions_dt['conId'][i] = positions_dt['contract'][i].conId


<ipython-input-43-a455ddaa679e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positions_dt['conId'][i] = positions_dt['contract'][i].conId


In [44]:
#positions_dt['contract'][2]

In [45]:
positions_dt = pd.merge( positions_dt, isin_dt[['conId', 'ISIN', 'LONG_COMP_NAME', 'PX_LAST' ]], on='conId', how='left')

In [46]:
positions_dt['Notional_Position'] = positions_dt['position'] * positions_dt['PX_LAST']

In [47]:
positions_dt

,account,contract,position,avgCost,conId,ISIN,LONG_COMP_NAME,PX_LAST,Notional_Position
0,DU5097959,"Stock(conId=29612246, symbol='STM', exchange='...",3208.0,39.098246,29612246,NL0000226223,STMicroelectronics NV,37.5900,120588.7200
1,DU5097959,"Stock(conId=29612183, symbol='RNO', exchange='...",3458.0,36.682372,29612183,FR0000131906,Renault SA,34.8875,120640.9750
2,DU5097959,"Stock(conId=29612178, symbol='KER', exchange='...",-187.0,678.132217,29612178,FR0000121485,Kering SA,651.9000,-121905.3000
3,DU5097959,"Stock(conId=35428035, symbol='ALO', exchange='...",4846.0,25.895870,35428035,FR0010220475,Alstom SA,24.5550,118993.5300
4,DU5097959,"Stock(conId=463647471, symbol='STLA', exchange...",7455.0,16.844827,463647471,NL00150001Q9,Stellantis NV,16.3060,121561.2300
5,DU5097959,"Stock(conId=29612172, symbol='CA', exchange='S...",-6926.0,17.787200,29612172,FR0000120172,Carrefour SA,17.8450,-123594.4700
6,DU5097959,"Forex('EURUSD', conId=12087792, localSymbol='E...",-1900.0,1.143864,12087792,NaN,NaN,NaN,NaN
7,DU5097959,"Stock(conId=29612232, symbol='GLE', exchange='...",3798.0,33.816787,29612232,FR0000130809,Societe Generale SA,31.7475,120577.0050
8,DU5097959,"Stock(conId=35052259, symbol='ENGI', exchange=...",-8688.0,14.393590,35052259,FR0010208488,Engie SA,14.2940,-124186.2720
9,DU5097959,"Stock(conId=500760943, symbol='DSY', exchange=...",-3007.0,42.173367,500760943,FR0014003TT8,Dassault Systemes SE,39.8700,-119889.0900


In [48]:
#sum(fill.commissionReport.commission for fill in ib.fills())

In [49]:
#util.df( ib.fills() )